In [ ]:
import os
import time
import open3d as o3d
import numpy as np
import torch
import torch.profiler
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from torch_geometric.typing import WITH_TORCH_CLUSTER
from pyg_pointnet2 import PyGPointNet2, PyGPointNet2NoColor
from pc_label_map import color_map

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Empty the CUDA cache
torch.cuda.empty_cache()

In [ ]:
# Models with x6 features including colours
model = PyGPointNet2(num_classes=13).to(device) # x6 model
model_file_path = "checkpoints/pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth" # Orignial transformed x6 model

In [ ]:
# Models with x3 features without colours
model = PyGPointNet2NoColor(num_classes=13).to(device) # x3 model
#model_file_path = "checkpoints/pointnet2_s3dis_transform_seg_x3_45_checkpoint.pth" # Orignial transformed x3 model
#model_file_path = "checkpoints/pointnet2_smartlab_sim_noise_seg_x3_50_checkpoint.pth" # Simulated transformed x3 model with noise
model_file_path = "checkpoints/smartlab_fine_tuning_transform_x3_50_20250831.pth" # Fine-tuned model


In [ ]:
# Load the checkpoint dictionary
checkpoint = torch.load(model_file_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)
model.eval()


In [ ]:
# Load point cloud files
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.pcd"
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.ply"
#pcd_path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\data\hexagon\Fira- 1001.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

In [ ]:
# Move the point cloud to its min(x,y,z) corner 
def move_to_corner(points):    
    # Find the minimum x, y, z
    min_xyz = points.min(axis=0)
    # Translate the point cloud so that the min corner becomes the origin
    moved_points = points - min_xyz
    
    return moved_points

moved_points = move_to_corner(np.array(pcd.points))
pcd.points = o3d.utility.Vector3dVector(moved_points)

# Downsample the point cloud with a voxel of 0.03
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

In [ ]:
# Calculate x features by normalization
def normalize_points_corner(points):
    min_vals = np.min(points, axis=0)
    shifted_points = points - min_vals    
    max_vals = np.max(shifted_points, axis=0)
    scale = max_vals.copy()    
    scale[scale == 0] = 1    
    normalized_points = shifted_points / scale

    return normalized_points

normalized = normalize_points_corner(np.array(downpcd.points))

In [13]:
# Extract coordinates and colors from the point cloud
down_points = torch.tensor(np.array(downpcd.points), dtype=torch.float32)  
down_colors = torch.tensor(np.array(downpcd.colors), dtype=torch.float32)
down_normalized = torch.tensor(normalized, dtype=torch.float32)

In [ ]:
# Data with x6 features including colours
# Concatenate coordinates and colors to form the input features
x = torch.cat([down_colors, down_normalized], dim=1) # 6 (x)
# Create a Data object with x (6 features) and pos (coordinates)
data = Data(x=x, pos=down_points)

In [ ]:
# Data with x3 features without colours
# Create a Data object with x (3 features) and pos (coordinates)
data = Data(x=down_normalized, pos=down_points)

In [15]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

num_workers = 20 # use os.cpu_count() to check number of cpu cores
batch_size = 32

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [ ]:
# Segmentation
model.eval()

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    record_shapes=True,
) as prof:

    with torch.no_grad():
        start_time = time.time()
        for data in custom_loader:
            data = data.to(device)
            with torch.amp.autocast("cuda"):
                predictions = model(data)
            labels = predictions.argmax(dim=-1)            
            unique_labels, label_counts = torch.unique(labels, return_counts=True)            
            result_labels = torch.stack((unique_labels, label_counts), dim=1).cpu()
            print("Label counts:")
            print(result_labels)
        end_time = time.time()
        print(f"Total inference time: {end_time - start_time:.4f} seconds")  
    
print(prof.key_averages().table(sort_by="cuda_time_total"))



Label counts:
tensor([[     0, 122024],
        [     1, 144159],
        [     2, 164724],
        [     3,      1],
        [     6,   5472],
        [     7,    820],
        [     8,   5002],
        [     9,      8],
        [    10,   7036],
        [    12, 417654]])
Total inference time: 264.5634 seconds
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     torch_cluster::fps         0.01%      31.368ms        

In [ ]:
# Assign predicted colors to the point cloud
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]
downpcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [39]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([downpcd])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [ ]:
# Save the point cloud to a file
save_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/labelled/Smartlab_finetune_noise_label_pointnet2_x3_0.03_20250831.ply"
o3d.io.write_point_cloud(save_path, downpcd)

True